In [1]:
# unset PYTHONPATH first
from ko_text import *

# 불러오기

In [2]:
token_df = pd.read_csv('Data/meta_morphs_final.csv', encoding = 'utf-8')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
token_df['Token'] = [token.split() for token in token_df['Token']]

In [3]:
token_df.head()

,Section,Text,Token,Num of Tokens
0,financial,\n\n\n텀블벅에서 크라우드 펀딩이 이뤄지고 있는 `아침달 시집`.\n\n ...,"[텀블벅, 크라, 우드, 펀딩, 이뤄지고, 아침, 시집, 많지, 않은, 금액, 으로...",263
1,economy,\n\n\n[사진 제공: 연합뉴스]\n\n 유류...,"[유류, 인하, 국제, 유가, 급락, 입어, 국내, 휘발유, 경유, 하락, 특히, ...",166
2,financial,부득이한 사정으로 매월 내는 보험료가 부담이 될 때 계약은 그대로 유지하면서 보험...,"[부득이, 사정, 매월, 내는, 보험료, 부담, 계약, 그대로, 유지, 보험료, 부...",314
3,estate,한때 `미분양의 늪`으로 통하던 경기도 파주시 부동산 시장이 달라지고 있다. 지난해...,"[한때, 미분, 하던, 경기도, 파주시, 부동산, 시장, 달라지고, 분양, 파주, ...",165
4,economy,\n\n\n인디고뱅크의 `미키인서울` 컬래버 맨투맨 <사진제공=월트디즈니코리아>\...,"[인디고, 뱅크, 미키, 서울, 컬래버, 투맨, 월트디즈니, 사의, 마스코트, 미키...",196


# embedding with FastText

In [4]:
from gensim.models import FastText

x_total_dataset = token_df['Token'].tolist()
model = FastText(x_total_dataset, size=100, window=5, min_count=5, workers=4, sg=1, min_n = 1)

## 학습을 위한 데이터 전처리

# Train Test Split

In [5]:
train_size = round(len(token_df) * 0.8)
np.random.seed(0)
train_index_ls = np.random.choice(token_df.index, train_size, replace = False)
test_index_ls = [x for x in token_df.index if not x in train_index_ls]

train_df = token_df.loc[train_index_ls]
test_df = token_df.loc[test_index_ls]


# Sampling

In [6]:
nlp = NLP()

In [7]:
# 한 label마다 학습할 단어의 수
train_batch_size = 3000
test_batch_size = 100

x_total_dataset = token_df['Token'].tolist()
x_train, y_train = nlp.oversample_batch(train_df['Token'], train_df['Section'], train_batch_size)
x_test, y_test =  nlp.undersample_batch(test_df['Token'],test_df['Section'], test_batch_size)

In [8]:
print(len(x_train))
Counter(y_train)

33000


Counter({'bio & tech': 3000,
         'business': 3000,
         'culture & art': 3000,
         'economy': 3000,
         'estate': 3000,
         'financial': 3000,
         'it': 3000,
         'politics': 3000,
         'society': 3000,
         'stock': 3000,
         'world': 3000})

# CBOW

In [9]:
X_train = []
for sent in x_train:
    temp_cbow = np.array([model.wv.__getitem__(token) if model.wv.__contains__(token) else np.zeros(1,100) for token in sent])
    X_train += [np.sum(temp_cbow, axis = 0).tolist()]

In [10]:
X_test = []
for sent in x_test:
    temp_cbow = np.array([model.wv.__getitem__(token) if model.wv.__contains__(token) else np.zeros(1,100) for token in sent])
    X_test += [np.sum(temp_cbow, axis=0).tolist()]

# Y 더미화

In [11]:
from collections import defaultdict

l2i_dict = defaultdict(lambda : len(l2i_dict))

y_train = [l2i_dict[y] for y in y_train]
y_test = [l2i_dict[y] for y in y_test]

# **Logistic Regression**

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [15]:
clf = LogisticRegression(solver = 'sag',
                         multi_class = 'multinomial')


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy : ', accuracy_score(y_pred, y_test))

Accuracy :  0.6554545454545454


/home/donghyungko/anaconda3/envs/fininsight_python3.5/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## **Linear SVM**

In [16]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline


In [17]:
clf = LinearSVC()


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy : ', accuracy_score(y_pred, y_test))

Accuracy :  0.3518181818181818
